# Python Object Introspection & Attribute Management
In Python, "everything is an object." This philosophy necessitates robust tools for **introspection**—the ability of code to examine the type, properties, and internal structure of objects at runtime. This module covers the essential built-in functions used for type checking, dynamic attribute management, and debugging.

---

## 1. Type Checking & Validation

While Python is dynamically typed, runtime type validation is often necessary for data processing pipelines or API development.

### `type(obj)`

Returns the class type of an object. It is strict and does not account for inheritance.

### `isinstance(obj, class_or_tuple)`

Checks if an object is an instance of a class or a subclass thereof. This is the **preferred** method for type checking in production code because it respects polymorphism (Liskov Substitution Principle).

**Key Difference:**

* `type(obj) == Class`: Checks for exact identity.
* `isinstance()`: Checks for inheritance compatibility.

#### Engineering Example: Handling Polymorphism

```python
class DataConnector:
    pass

class SQLConnector(DataConnector):
    pass

class APIConnector(DataConnector):
    pass

def connect_to_source(connector):
    # BAD: Strict type checking fails for subclasses
    # if type(connector) == DataConnector: ...
    
    # GOOD: Supports SQLConnector and APIConnector
    # Also supports checking against a tuple of types
    if isinstance(connector, (SQLConnector, APIConnector)):
        print(f"Connection established with {type(connector).__name__}")
    else:
        raise TypeError("Invalid connector type provided")

db = SQLConnector()
connect_to_source(db)  # Validates successfully

```

---

## 2. Dynamic Attribute Management

Python allows you to access, check, and retrieve object attributes dynamically using strings. This is the backbone of many frameworks (like ORMs) and plugin systems.

### `hasattr(obj, name)`

Returns `True` if the string `name` is an attribute or method of `obj`.

### `getattr(obj, name, default)`

Returns the value of the named attribute of `obj`.

* **Critical Best Practice:** Always provide the `default` argument. If the attribute is missing and no default is provided, Python raises an `AttributeError`, which can crash your application.

#### Engineering Example: Feature Toggles

Instead of hardcoding method calls, you can invoke methods based on configuration strings (Dynamic Dispatch).

```python
class ImageProcessor:
    def filter_blur(self):
        return "Applying Blur"
    
    def filter_sharpen(self):
        return "Applying Sharpen"

def apply_effect(processor, effect_name):
    # Construct method name dynamically (e.g., 'filter_blur')
    method_name = f"filter_{effect_name}"
    
    # Check existence before access
    if hasattr(processor, method_name):
        # Get the method reference safely
        effect_method = getattr(processor, method_name)
        return effect_method()
    else:
        # Graceful fallback using getattr default isn't applicable for methods 
        # that need execution, so we handle logic here.
        return f"Effect '{effect_name}' not implemented."

proc = ImageProcessor()
print(apply_effect(proc, "sharpen"))  # Output: Applying Sharpen
print(apply_effect(proc, "sepia"))    # Output: Effect 'sepia' not implemented.

```

---

## 3. Object Identity: `id()`

The `id()` function returns the "identity" of an object. In CPython (the standard Python implementation), this is the **memory address** of the object.

* **Usage:** It is rarely used in high-level logic but is critical for debugging mutable state issues (e.g., checking if two variables reference the exact same list in memory).
* **Equality vs. Identity:** `==` checks value equality; `is` checks identity (based on `id()`).

```python
# List Aliasing (Reference Copy)
config_a = ["debug=True", "port=8080"]
config_b = config_a  # Both point to the same object

# List Copying (Value Copy)
config_c = config_a[:] # Creates a new object in memory

print(f"A vs B (Same Object?): {id(config_a) == id(config_b)}") # True
print(f"A vs C (Same Object?): {id(config_a) == id(config_c)}") # False

```

---

## 4. Object Inspection: `dir()`

`dir()` attempts to return a list of valid attributes for the object. It is a debugging aid, not a strict interface definition.

* **Without arguments:** Returns names in the current local scope.
* **With object:** Returns attributes of the object (including magic methods like `__init__`).

#### Engineering Example: Inspecting Third-Party Libraries

When documentation is sparse, `dir()` helps uncover available methods.

```python
import json

# What can the json module actually do?
attributes = dir(json)

# Filter for public methods (exclude magic methods starting with __)
public_api = [attr for attr in attributes if not attr.startswith("_")]
print(f"JSON Module Public API: {public_api}")
# Output: ['dump', 'dumps', 'load', 'loads', ...]

```

---

## 5. Representation: `repr()` vs `str()`

While `str()` provides a readable string for end-users, `repr()` provides the "official" string representation, primarily for developers.

* **Contract:** A good `repr()` should ideally be a valid Python expression that could recreate the object (e.g., `Point(x=1, y=2)` instead of `Point object at 0x...`).
* **Fallback:** If a class doesn't define `__str__`, Python uses `__repr__` as a fallback.

```python
class ServerConfig:
    def __init__(self, host, port):
        self.host = host
        self.port = port
    
    def __repr__(self):
        # Unambiguous representation for debugging
        return f"ServerConfig(host='{self.host}', port={self.port})"
    
    def __str__(self):
        # Readable format for logging
        return f"Server running on {self.host}:{self.port}"

cfg = ServerConfig("127.0.0.1", 5000)

print(str(cfg))   # Output: Server running on 127.0.0.1:5000
print(repr(cfg))  # Output: ServerConfig(host='127.0.0.1', port=5000)

```